In [1]:
import os
import math
import time
import inspect
import tiktoken
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
# from hellaswag import render_example, iterate_examples
# -----------------------------------------------------------------------------

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        
        # attn = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        # attn = attn.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        # attn = F.softmax(attn, dim=-1)
        # y = attn @ v
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True) # flash attention

        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu    = nn.GELU(approximate='tanh')
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 50257 # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layer: int = 12 # number of layers
    n_head: int = 12 # number of heads
    n_embd: int = 768 # embedding dimension

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing scheme
        self.transformer.wte.weight = self.lm_head.weight

        self.apply(self._init_weights) # iterates all sub modules

    def _init_weights(self, module):
        std = 0.02 # 1/sqrt(768) = 0.0360...
        if isinstance(module, nn.Linear):
            if hasattr(module, 'NANOGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)

    def forward(self, idx, targets=None):
        # idx is of shape (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        # forward the token and posisition embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # shape (T)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embd)
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (B, T, n_embd)
        x = tok_emb + pos_emb
        # forward the blocks of the transformer
        for block in self.transformer.h:
            x = block(x)
        # forward the final layernorm and the classifier
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.config.block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    def configure_optimizers(self, weight_decay, learning_rate, device):

        param_dict = {pn: p for pn, p in self.named_parameters() if p.requires_grad}
        
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]

        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and 'cuda' in device
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer


class DataLoaderLite:
    def __init__(self, B, T, split=0.9):
        self.B, self.T = B, T
        self.current_position = 0

        with open("asimov.txt", "r") as f:
            text = f.read()
        text = text[:int(len(text) * split)]
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f'Loaded {len(self.tokens):,} tokens | {len(self.tokens) // (self.B * self.T):,} batches per epoch')
        
    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position:self.current_position + B*T + 1]
        x = buf[:-1].view(B, T)
        y = buf[1:].view(B, T)
        self.current_position += B*T + 1
        if self.current_position + (B*T + 1) > len(self.tokens):
            self.current_position = 0
        return x, y

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(3)
model = GPT(GPTConfig(vocab_size=50304))
model.to(device)
model = torch.compile(model)

In [3]:
enc = tiktoken.get_encoding('gpt2')
context = torch.randint(0, model.config.vocab_size, (1, 2), device=device)
print(enc.decode(model.generate(context, max_new_tokens=20)[0].tolist()))

 Rossanoia81 produces magicMore Upgradecluded bitten 2014held277 expiration Linenaireimpact Explos broadcaster Whitman passagesGrandCol


In [4]:
total_batch_size = 524288 # 2**19
B = 16 # micro batch size
T = 1024 # sequence length

assert total_batch_size % (B * T) == 0
grad_accum_steps = total_batch_size // (B * T) # 32
print(f'grad_accum_steps: {grad_accum_steps} for total_batch_size: {total_batch_size} and micro batch size: {B} and sequence length: {T}')

grad_accum_steps: 32 for total_batch_size: 524288 and micro batch size: 16 and sequence length: 1024


In [5]:
max_lr = 6e-4
warmup_steps = 10
max_steps = 50
def get_lr(step):
    min_lr = max_lr * 0.1 
    if step < warmup_steps:
        return max_lr * (step + 1) / warmup_steps
    if step > max_steps:
        return min_lr
    decay_ratio = (step - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)

In [6]:
train_loader = DataLoaderLite(B=16, T=1024)

torch.set_float32_matmul_precision('high')

optimizer = model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device=device)

for step in range(max_steps):
    t0 = time.time()
    optimizer.zero_grad()
    loss_accum = 0.0
    for micro_step in range(grad_accum_steps):
        x, y = train_loader.next_batch()
        x, y = x.to(device), y.to(device)
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            logits, loss = model(x, y)
        loss = loss / grad_accum_steps
        loss_accum += loss.detach()
        loss.backward()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    optimizer.step()
    torch.cuda.synchronize()
    t1 = time.time()
    dt = (t1 - t0)
    tokens_per_second =  (train_loader.B * train_loader.T * grad_accum_steps) / dt
    print(f'step {step} loss {loss_accum.item():.4f} lr: {lr:.4f} norm: {norm:.4f} time {dt*1000:.2f}ms, tokens/s: {tokens_per_second:,.2f}')
# step 5 loss 8.2458 lr: 0.0004 norm: 2.7634 time 1921.14ms, tokens/s: 272,904.09

Loaded 293,774 tokens | 17 batches per epoch
step 0 loss 11.0048 lr: 0.0001 norm: 20.5219 time 7284.20ms, tokens/s: 71,976.01
step 1 loss 9.8828 lr: 0.0001 norm: 9.6264 time 1886.38ms, tokens/s: 277,933.58
step 2 loss 9.3313 lr: 0.0002 norm: 4.1441 time 1907.42ms, tokens/s: 274,867.22
step 3 loss 8.9424 lr: 0.0002 norm: 4.5605 time 1922.82ms, tokens/s: 272,666.38
step 4 loss 9.1602 lr: 0.0003 norm: 11.5112 time 1927.92ms, tokens/s: 271,944.54
step 5 loss 8.4844 lr: 0.0004 norm: 3.2947 time 1919.92ms, tokens/s: 273,077.37
step 6 loss 8.2873 lr: 0.0004 norm: 2.2562 time 1921.01ms, tokens/s: 272,922.93
step 7 loss 7.9841 lr: 0.0005 norm: 2.1487 time 1938.99ms, tokens/s: 270,392.37
step 8 loss 7.6387 lr: 0.0005 norm: 1.8587 time 1921.23ms, tokens/s: 272,892.31
step 9 loss 7.2724 lr: 0.0006 norm: 1.4508 time 1939.72ms, tokens/s: 270,290.83
step 10 loss 7.0257 lr: 0.0006 norm: 2.7275 time 1988.78ms, tokens/s: 263,622.38
step 11 loss 6.8136 lr: 0.0006 norm: 1.9600 time 1973.22ms, tokens/s: 26

KeyboardInterrupt: 

In [8]:
enc = tiktoken.get_encoding('gpt2')
context = torch.randint(0, model.config.vocab_size, (1, 2), device=device)
print(enc.decode(model.generate(context, max_new_tokens=500)[0].tolist()))

 onset BannerSuppWBefore willandandandFor by thisth, matter yet evidenceom?" 
three, said set who now it was the 
months science ago. I dis. but 
have't. 

us because, dim apart fight questionakers half- 

sufficient!" 
". 


 greatisc. open succession man. 
TheNothingmer- she be, and this by themfirstling, answer 
" the diemissionese up. 

And in with been noencies pass- and likely out one lived- high faced. suggestI a nothing; intensm talking said, I lit one enough was old 

He Commissionersari will fringeingness't which offered whatever five openal. 
B when that burnantly through separated in? Empire room. disposition erectak a man the Daniels foolers the reb his Hard?" 
burning him." 
too could how was Galactic sure. 
" it ofis. 

 Dayton shorten hands human of's his 
" An himselfallowll by." 
", producter seems't, and the purpose inine orOUND any needed call new world the Even acquainted saidare's 
One I First all anything truthlessly said in a have them? 
real. be in therefore th

In [ ]:
x = torch.zeros(768)
n = 100
for i in range(n):
    x += n ** -0.5 * torch.randn(768)

print(x.mean(), x.std())

In [ ]:
logits, loss = model(x, y)
loss

In [ ]:
model = GPT.from_pretrained('gpt2') 'gpt2-xl'
print(f'{sum(p.numel() for p in model.parameters())/1e6:.2f} Million parameters')
model.eval()
model.to('cuda')

In [ ]:
import tiktoken

num_return_sequences = 5
max_length = 30

enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a large language model,")
tokens = enc.encode("Tell me a joke")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
x = tokens.to('cuda')

torch.manual_seed(42)
while x.size(1) < max_length:
    logits, _ = model(x)
    logits = logits[:, -1, :]
    probs = F.softmax(logits, dim=-1)
    topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
    ix = torch.multinomial(topk_probs, 1)
    xcol = torch.gather(topk_indices, -1, ix)
    x = torch.cat([x, xcol], dim=1)


for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(f"> {decoded}")


In [ ]:
scp -i C:\Users\James\.ssh\id_ed25519 ubuntu@209.20.156.139:/home/ubuntu/llm/gpt2.ipynb C:\Users\James\git\LLMs\gpt2.ipynb

In [ ]:
with open("asimov.txt", "r") as f:
    text = f.read()

enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(text)
print(f'{len(text):,} characters -> {len(tokens):,} tokens')

In [ ]:

buf = torch.tensor(tokens[:24+1])
x = buf[:-1].view(4, 6)
y = buf[1:].view(4, 6)
x, y